In [1]:
source_file = "/Users/varunpasad/projects/envee/addons/data/Envee-ERP Export Sheet.xlsx"
images_dir = "/Users/varunpasad/projects/envee/addons/data/images/"
product_file = ""
# mrp_file = "/Users/varunpasad/projects/odoo/tcuts/data/marinades/mrp.bom.csv"
# mrp_line_file = "/Users/varunpasad/projects/odoo/tcuts/data/marinades/mrp.bom.csv"

# source_file = "/Users/varunpasad/projects/odoo/wastage/import/wastage.xlsx"
mrp_file = "/Users/varunpasad/projects/envee/addons/data/mrp.bom.csv"
mrp_line_file = "/Users/varunpasad/projects/envee/addons/data/mrp.bom.line.csv"

In [9]:
# load the sheet
import pandas as pd
import numpy as np
import os
import glob
import base64

class MarinadeCatalogGen:
    ING_NAME = "ingredients"
    RECEIPE_NAME = "receipe"

    def __init__(self):
        #self.sheet_name = "/Users/varunpasad/Downloads/export_marinades - FILLED.xlsx"
        self.sheet_name = source_file
        self.ing_df = None
        self.int_df = None
        self.receipe_df = None
    
    def fetch_products(self):
        sheet_db = pd.read_excel(self.sheet_name, sheetname=[
            self.ING_NAME,
            self.RECEIPE_NAME
        ])
       
        self.ing_df = sheet_db[self.ING_NAME]
        self.receipe_df = sheet_db[self.RECEIPE_NAME]

        
class ProductTemplGen:

    def __init__(self, df):
        self.df = df
    
    def reformat_sheet(self):
        """Format the sheet to odoo understandable cols"""
        self.df = self.df.reset_index()
        self.df = self.df.rename(columns={
            'sku': 'default_code',
            'cost':'standard_price',
            'sell': 'lst_price'
        })
        self.df = self.df.set_index('default_code')
    
    def add_default_cols(self):
        """Adds necessary cols
        
        type, tracking, active (based on dep) and finally generated XID
        """
        # external id
        self.df['type']  = "Stockable Product"
#         self.df['tracking']  = "lot"
        self.df = self.df.reset_index().apply(self.build_row, axis=1)

        # convert _DEP to inactive in odoo and build the ID
        self.df['active'] = np.where(
            self.df.default_code.str.endswith("_DEP"), "FALSE", "TRUE")
        self.df.default_code = self.df.default_code.str.replace(
            r"^(.*)_DEP$", r"\1")

        self.df = self.df.set_index("default_code")
        self.df['id'] = "tcuts.product_" + self.df.index
        # drop off anything that does not have category
        self.df = self.df[~self.df['categ_id/id'].isnull()]

        # set active/inactive
        # set plu code
        self.df['plu_code'] = range(100,100 + len(self.df))
    
    def tracking(self, tracking):
        tracking_map = {
            "lot": "lot",
            "serial": "serial",
            "none": "none"
        }
   
        return tracking_map[tracking]

    def category(self, category):
        # GEnerate catalog
        CAT_MAP = {
            "Ingredients": "tcuts.product_category_ingredients",
            "Starters":  "tcuts.product_category_marinades",
            "Gravies":  "tcuts.product_category_gravies",
            "Rice & Roti": "tcuts.product_category_rar",
            "Rolls & Pops": "tcuts.product_category_rp",
            "Manufactured Materials": "tcuts.product_category_mraw",
            "Raw Materials": "tcuts.product_category_raw",
            "Purchased Materials": "tcuts.product_category_praw"
        }
        return CAT_MAP[category]

    def pos_category(self, category):
        CAT_MAP = {
            "tcuts.product_category_marinades": "tcuts.pos_category_marinades",
            "tcuts.product_category_gravies": "tcuts.pos_category_gravies",
            "tcuts.product_category_rar": "tcuts.pos_category_rar",
            "tcuts.product_category_rp": "tcuts.pos_category_rp"
        }
        
        try:
            return CAT_MAP[category]
        except KeyError:
            return None
    
    def uom(self, uom):
        UOM_MAP = {
            "kg": "product.product_uom_kgm",
            "unit": "product.product_uom_unit",
            "2 pieces per pack": "tcuts.product_uom_unit2",
            "3 pieces per pack": "tcuts.product_uom_unit3",
            "6 pieces per pack": "tcuts.product_uom_unit6",
        }
        return UOM_MAP[uom]
    
    def taxes_id(self, category):
        return None
        TAX_MAP = {
            "tcuts.product_category_marinades": 'l10n_in.1_sgst_sale_5',
            "tcuts.product_category_gravies": 'l10n_in.1_sgst_sale_5',
            "tcuts.product_category_rar": 'l10n_in.1_sgst_sale_5',
            "tcuts.product_category_rp": 'l10n_in.1_sgst_sale_5'
        }
        try:
            return TAX_MAP[category]
        except KeyError:
            return None

    
    def build_row(self, row):
        """Generate the row data
        such as salelable, pruchasable etc
        """
        sku = row['default_code']
#         is_raw = self.is_raw_material(sku)

        # SKU
        row['purchase_ok'] = row.purchase_ok
        row['sale_ok'] = row.sale_ok
        
        row['categ_id/id'] = self.category(row.category)
        row['pos_categ_id/id'] = self.pos_category(row['categ_id/id'])
        row['tracking'] = self.tracking(row.tracking)

        row['life_time'] = row.expiry
        row['use_time'] = row.expiry - 2
        row['alert_time'] = row.expiry - 2
        row['removal_time'] = row.expiry -1
        
        # no taxes
        row['uom_po_id/id'] = self.uom(row.uom)
        row['uom_id/id'] = self.uom(row.uom)
        
        row['taxes_id/id'] = self.taxes_id(row['categ_id/id'])
        row['supplier_taxes_id'] = None
        
        # finally add the image
        image_file = os.path.join(images_dir, "{}.JPG".format(row.default_code))
        if os.path.exists(image_file):
            print(image_file)
            with open(image_file, "rb") as fh:
                row['image'] = base64.b64encode(fh.read())
            

        return row

    def generate_catalog(self):
        self.reformat_sheet()
        self.add_default_cols()
    
    def to_csv(self, path):
        booleanDictionary = {True: 'TRUE', False: 'FALSE'}
        self.df.purchase_ok = self.df.purchase_ok.map(booleanDictionary)
        self.df.sale_ok = self.df.sale_ok.map(booleanDictionary)
        self.df.available_in_pos = self.df.available_in_pos.map(booleanDictionary)

        self.df.reset_index().to_csv(
            path,
            index=False,
            columns=[
                "id", "default_code", "name", "type", "tracking", "categ_id/id",
                "uom_po_id/id", "uom_id/id", "purchase_ok", "sale_ok",
                "use_time", "alert_time", "removal_time", "life_time",
                "taxes_id/id", "supplier_taxes_id", "standard_price", "lst_price",
                "active", "plu_code", "available_in_pos", "pos_categ_id/id",
                "image"
            ]
            )
 
class BomGen:
    def __init__(self, df, variant=True):
        """sku, parent_sku,weight"""
        self.df = df
        self.variant = variant
    
    def uom(self, uom):
        UOM_MAP = {
            "kg": "product.product_uom_kgm",
            "unit": "product.product_uom_unit",
            "2 pieces per pack": "tcuts.product_uom_unit2",
            "3 pieces per pack": "tcuts.product_uom_unit3",
            "6 pieces per pack": "tcuts.product_uom_unit6",
        }
        return UOM_MAP[uom]

    def routing_id(self, sku):
        if sku.startswith("RM_"):
            return "tcuts.mrp_routing_main"
        
        return None
    
    def build_row(self, row):
        sku = row['sku']
        row['product_uom_id/id'] = self.uom(row.uom)
        row['routing_id/id'] = self.routing_id(sku)
        return row
    
    def reformat_cols(self):
        self.df = self.df.reset_index()[['sku', 'weight', 'variant', 'uom']]

        self.df = self.df.rename(columns={
            "weight": "product_qty"
        })
        
        # add active flag.
        self.df['active'] = np.where(
            self.df.sku.str.endswith("_DEP"), "FALSE", "TRUE")
        
        # And remove the _DEP from the name.
        self.df.sku = self.df.sku.str.replace(r"^(.*)_DEP$", r"\1")
        
        # generate product ID:
        self.df['product_tmpl_id/id'] = "tcuts.product_" + self.df.sku

        # generate Ids
        self.df['id'] = "tcuts.mrp_bom_" + self.df.sku
        if self.variant:
            self.df.id = self.df.id + "_" + self.df.variant.astype(int).astype(str)
            del self.df['variant']
        
        # inject addl cols
        self.df['type'] = "Manufacture this product"
#         self.df['product_uom_id/id'] = 'product.product_uom_kgm'
        
        self.df = self.df.apply(self.build_row, axis=1)
    
    
    def generate_bom(self):
        # rolling fill
        self.df = self.df.fillna(method='ffill')
        self.reformat_cols()
        self.df = self.df.drop_duplicates('id')
        
        # now on to bom.line
    
    def to_csv(self, path):
        self.df.reset_index().to_csv(
            path,
            index=False,
            columns=["id", "product_tmpl_id/id", "product_qty", "type", "active", "product_uom_id/id", "routing_id/id"]
        )
        
class BomLineGen:
    def __init__(self, df):
        """sku,variant,weight,name,parent_sku,qty"""
        self.df = df
    
    def uom(self, uom):
        UOM_MAP = {
            "kg": "product.product_uom_kgm",
            "unit": "product.product_uom_unit",
            "2 pieces per pack": "tcuts.product_uom_unit2",
            "3 pieces per pack": "tcuts.product_uom_unit3",
            "6 pieces per pack": "tcuts.product_uom_unit6",
        }
        return UOM_MAP[uom]
    
    def operation_id(self, sku):
        if sku.startswith("RM_"):
            return "tcuts.mrp_routing_workcenter_pack"
        
        return None
        
    
    def build_row(self, row):
        sku = row['sku']
        row['product_uom_id/id'] = self.uom(row.line_uom)
        row['operation_id/id'] = self.operation_id(sku)
        return row
    
    
    def reformat_cols(self):
        
        # remove dep
        self.df.sku = self.df.sku.str.replace(r"^(.*)_DEP$", r"\1")
        self.df.parent_sku = self.df.parent_sku.str.replace(r"^(.*)_DEP$", r"\1")
        
        # generat the BOM id of the SKU
        self.df['bom_id/id'] = "tcuts.mrp_bom_" + self.df.sku
        self.df['bom_id/id'] = \
            self.df['bom_id/id'] + "_" + self.df.variant.astype(int).astype(str)
        
        # generate product ID for the BOM(parent_Sku)
        self.df['product_id/id'] = "tcuts.product_" + self.df.parent_sku
    
        # finally the bom line ID:
        self.df['id'] = \
            "tcuts.bom_line_" + self.df.sku + "_" + self.df.variant.astype(int).astype(str) \
            + "_" + self.df.recipe_index.astype(int).astype(str)
        
        # add cons cols
#         self.df['product_uom_id/id'] = 'product.product_uom_kgm'
        self.df['product_qty'] = self.df.qty
        
        self.df = self.df.apply(self.build_row, axis=1)
        
        self.df = self.df[['id', 'product_id/id', 'product_qty', 'product_uom_id/id', 'bom_id/id', 'operation_id/id']]

    def generate_bom(self):
        # rolling fill
        self.df = self.df.fillna(method='ffill')
        self.reformat_cols()
        
    def to_csv(self, path):
        self.df.reset_index().to_csv(
            path,
            index=False,
            columns=['id', 'product_id/id', 'product_qty', 'product_uom_id/id', 'bom_id/id', 'operation_id/id']
        )
    
        


In [10]:
# build the base sheet
gen = MarinadeCatalogGen()
gen.fetch_products()

prod = ProductTemplGen(gen.ing_df)
prod.generate_catalog()
prod.to_csv("/Users/varunpasad/projects/envee/addons/data/product.template.csv")

bomgen = BomGen(gen.receipe_df)
bomgen.generate_bom()
bomgen.to_csv(mrp_file)

bomlinegen = BomLineGen(gen.receipe_df)
bomlinegen.generate_bom()
bomlinegen.to_csv(mrp_line_file)


/Users/varunpasad/projects/envee/addons/data/images/MRT_CHTD_CHK_65.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_CHTD_PRAWNS_65.JPG
/Users/varunpasad/projects/envee/addons/data/images/RP_CHK_POPCORN.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_CHILLI_GARLIC_PRAWNS.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_CLASSIC_BBQ_WINGS.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_KEEMA_TIKKA.JPG
/Users/varunpasad/projects/envee/addons/data/images/RP_KOLA_URUNDAI.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_LEMONE_CHK.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_MALAI_CHK.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_MARINA_ROAST.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_CHK_65_BL.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_SEEKH_KEBAB.JPG
/Users/varunpasad/projects/envee/addons/data/images/MRT_CHTD_CHK_WINGS.JPG
/Users/varunpasad/projects/envee/addons/data/ima